# Clustering StackOverflow Q&A

EPFL Big Data Analysis Week 2 Assignment
https://www.coursera.org/learn/scala-spark-big-data/home/info

"The overall goal of this assignment is to implement a distributed k-means algorithm which clusters posts on the popular question-answer platform StackOverflow according to their score. Moreover, this clustering should be executed in parallel for different programming languages, and the results should be compared.

The motivation is as follows: StackOverflow is an important source of documentation. However, different user-provided answers may have very different ratings (based on user votes) based on their perceived value. Therefore, we would like to look at the distribution of questions and their answers. For example, how many highly-rated answers do StackOverflow users post, and how high are their scores? Are there big differences between higher-rated answers and lower-rated ones?"

Data file download link: http://alaska.epfl.ch/~dockermoocs/bigdata/stackoverflow.csv

**WORK IN PROGRESS**

In [1]:
import time

# Credits to Fahim Sakri 
# Source (https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d)
# An annotation for timing a python function
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ("%r  %2.2f ms" % (method.__name__, (te - ts) * 1000))
        return result
    return timed

from post import Post

In [2]:
## Setup
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np

spark = SparkSession \
    .builder \
    .appName("EPFL Wk2 Assignment") \
    .getOrCreate()
        
spark.conf.set("spark.executor.instances", 1)
spark.conf.set("spark.executor.cores", 1)
spark.conf.set("spark.cores.max", 1)
spark.sparkContext.addPyFile('post.py')

# Create RDD
data = spark.read.csv('/data/epfl-big-data-analysis/stackoverflow.csv', header=False, inferSchema=True)
data = data.withColumnRenamed("_c0", "post_type_id") # type 1 = question, type 2 = answer
data = data.withColumnRenamed("_c1", "id")
data = data.withColumnRenamed("_c2", "acceptedAnswerId")
data = data.withColumnRenamed("_c3", "parentId")
data = data.withColumnRenamed("_c4", "score")
data = data.withColumnRenamed("_c5", "tag")

#data = pd.read_csv('/data/epfl-big-data-analysis/stackoverflow.csv', 
#                   names=["post_type_id", "id", "acceptedAnswerId", "parentId", "score", "tag"],
#                   dtype={'post_type_id': np.int64, 'score': np.float16})
data_size = data.count()

In [3]:
data.select('tag').distinct().collect()

[Row(tag='C#'),
 Row(tag='JavaScript'),
 Row(tag='Perl'),
 Row(tag=None),
 Row(tag='C++'),
 Row(tag='Groovy'),
 Row(tag='Objective-C'),
 Row(tag='CSS'),
 Row(tag='MATLAB'),
 Row(tag='Haskell'),
 Row(tag='Scala'),
 Row(tag='Clojure'),
 Row(tag='PHP'),
 Row(tag='Ruby'),
 Row(tag='Python'),
 Row(tag='Java')]

In [10]:
## Helper methods for clustering
def euclideanDistanceSum(x1, y1, x2, y2):
    dist = np.sqrt(np.square(np.subtract(px, x)) + np.square(np.subtract(py, y)))
    return np.sum(dist)

def findClosest(px, py, x, y):
    dist = np.sqrt(np.square(np.subtract(px, x)) + np.square(np.subtract(py, y)))
    return np.argmin(dist)

#def averageVectors(x1, x2):
#return np.av

In [77]:
posts = spark.sparkContext.parallelize(data.head(100)).filter(lambda p: p.tag is not None or p.post_type_id == 2)
data_size = 100                                                 
#langs = ["JavaScript", "Java", "PHP", "Python", "C#", "C++", "Ruby", "CSS",
#"Objective-C", "Perl", "Scala", "Haskell", "MATLAB", "Clojure", "Groovy"]
print(posts.take(1))
#posts.filter(lambda p: p.post_type_id == 1)
langs = posts.map(lambda p: p.tag).distinct().collect()
print(langs)

[Row(post_type_id=1, id=27233496, acceptedAnswerId=None, parentId=None, score=0, tag='C#')]
[None, 'Java', 'Python', 'JavaScript', 'CSS', 'C++', 'Objective-C', 'Ruby', 'PHP', 'C#']


### Step 1 Preparation - Grouped posts by question
First we use the map function to create kv pairs for each type of posts namely questions and answers.  
Then a join operation is used for merging the two datasets.  A dataset `RDD[(QID, Iterable(Question, Answer))]` should be useful, the key is the ID of the question post and the values is a collection of tuple (Question, Answer).

In [78]:
questions = posts.filter(lambda p: p.post_type_id == 1).map(lambda p: (p.id, p))
answers = posts.filter(lambda p: p.post_type_id == 2).map(lambda p: (p.parentId, p))
grouped = questions.join(answers).groupByKey() # Use inner join to exclude posts with no answers
print(questions.take(1))
print(answers.take(1))
print(grouped.take(2))

[(27233496, Row(post_type_id=1, id=27233496, acceptedAnswerId=None, parentId=None, score=0, tag='C#'))]
[(5484340, Row(post_type_id=2, id=5494879, acceptedAnswerId=None, parentId=5484340, score=1, tag=None))]
[(21984912, <pyspark.resultiterable.ResultIterable object at 0x7fb31d7f8860>), (3627793, <pyspark.resultiterable.ResultIterable object at 0x7fb31d816630>)]


### Step 2 Calculate maximum answer score for each question
Produce a set of key-value pairs - Key of the pair is the question and value should be the maximum answer score of the question.  The output is an `RDD[(Posting, Int)]`

In [79]:
def post_max_scores(iterable):
    max_score = -1
    for pair in iterable:
        question = pair[0]
        answer_score = pair[1].score
        if answer_score > max_score:
            max_score = answer_score
        return (question, max_score)

post_scores = grouped.values().map(post_max_scores) #(post_max_score)
print(post_scores.take(2))

[(Row(post_type_id=1, id=21984912, acceptedAnswerId=None, parentId=None, score=0, tag='Java'), 0), (Row(post_type_id=1, id=3627793, acceptedAnswerId=None, parentId=None, score=45, tag='Python'), 31)]


### Step 3 Create vectors for clustering
Prepare the vectors as an input for clustering.  

<br/>
Index of the language (in the langs list) multiplied by the `langSpread` factor.

The highest answer score (computed above).

The `langSpread factor` is provided (set to 50000). Basically, it makes sure posts about different programming languages have at least distance 50000 using the distance measure provided by the euclideanDist function. You will learn later what this distance means and why it is set to this value. The output is `RDD[(Int, Int)]`

In [85]:
langSpread = 50000

vectors = post_scores.map(lambda s: (langs.index(s[0].tag)*langSpread, s[1]))
print(vectors.count())
print(vectors.take(1))

25
[(50000, 0)]


In [93]:
# K-means parameter: Number of clusters
kmeansKernels = 3

# K-means parameter: Convergence criteria
kmeansEta = 20

# K-means parameter: Maximum iterations
kmeansMaxIterations = 120

sample_ratio = 1/len(langs)#/len(langs)

print(sample_ratio)
fractions = dict([(langs.index(lang)*langSpread, 0.1) for lang in langs])
sample_vectors = vectors.sampleByKey(False, fractions).head(kmeansKernels)
print(fractions)
print(vectors.count())
print(sample_vectors.count())
k.take(1)


0.1
{0: 0.1, 50000: 0.1, 100000: 0.1, 150000: 0.1, 200000: 0.1, 250000: 0.1, 300000: 0.1, 350000: 0.1, 400000: 0.1, 450000: 0.1}
25
4


[450000]

 `val newMeans = means.clone # Copy the means
    # For each vector find the closest centroids, once a cluster is formed, calculate average
    val centroids = vectors.map {
      v => (findClosest(v, means), v)
    }.groupByKey().mapValues {
      vs => averageVectors(vs)
    }.collect()
    
    # Replace with new mean
    for ((i, centroid) <- centroids) {
      newMeans.update(i, centroid)
    }

    val distance = euclideanDistance(means, newMeans)

    if (debug) {
      println(s"""Iteration: $iter
                 |  * current distance: $distance
                 |  * desired distance: $kmeansEta
                 |  * means:""".stripMargin)
      for (idx <- 0 until kmeansKernels)
        println(f"   ${means(idx).toString}%20s ==> ${newMeans(idx).toString}%20s  " +
          f"  distance: ${euclideanDistance(means(idx), newMeans(idx))}%8.0f")
    }

    if (converged(distance)) {
      newMeans
    } else if (iter < kmeansMaxIterations) {
      kmeans(newMeans, vectors, iter + 1, debug)
    } else {
      println("Reached max iterations!")
      newMeans
    }`

Based on these initial means, and the provided variables converged method, implement the K-means algorithm by iteratively:

pairing each vector with the index of the closest mean (its cluster);

computing the new means by averaging the values of each cluster.

To implement these iterative steps, use the provided functions findClosest, averageVectors, and euclideanDistance.

Note 1:

In our tests, convergence is reached after 44 iterations (for langSpread=50000) and in 104 iterations (for langSpread=1), and for the first iterations the distance kept growing. Although it may look like something is wrong, this is the expected behavior. Having many remote points forces the kernels to shift quite a bit and with each shift the effects ripple to other kernels, which also move around, and so on. Be patient, in 44 iterations the distance will drop from over 100000 to 13, satisfying the convergence condition.

If you want to get the results faster, feel free to downsample the data (each iteration is faster, but it still takes around 40 steps to converge):

val scored  = scoredPostings(grouped).sample(true, 0.1, 0)
However, keep in mind that we will test your assignment on the full data set. So that means you can downsample for experimentation, but make sure your algorithm works on the full data set when you submit for grading.

Note 2:

The variable langSpread corresponds to how far away are languages from the clustering algorithm's point of view. For a value of 50000, the languages are too far away to be clustered together at all, resulting in a clustering that only takes scores into account for each language (similarly to partitioning the data across languages and then clustering based on the score). A more interesting (but less scientific) clustering occurs when langSpread is set to 1 (we can't set it to 0, as it loses language information completely), where we cluster according to the score. See which language dominates the top questions now?

In [ ]:
 https://github.com/seahrh/stackoverflow-spark
        
 val lines   = sc.textFile("src/main/resources/stackoverflow/stackoverflow.csv")  

  val raw     = rawPostings(lines)  

  val grouped = groupedPostings(raw)  

  val scored  = scoredPostings(grouped)  

  val vectors = vectorPostings(scored)
    
    lines: the lines from the csv file as strings

raw: the raw Posting entries for each line

grouped: questions and answers grouped together

scored: questions and scores

vectors: pairs of (language, score) for each question